In [89]:
import pandas as pd
from scipy import integrate
from datetime import datetime, timedelta
import matplotlib.pyplot as

neutron_pvs = ['RAD:Berthold:TotalNeutronRate', 'RAD:ELSE:Neutron', 'RAD:Thermo1:Neutron',\
    'RAD:Thermo2:Neutron', 'RAD:Thermo3:Neutron', 'RAD:Thermo4:Neutron', 'RAD:Thermo5:Neutron',\
        'RAD:Thermo6:Neutron', 'RAD:Thermo7:Neutron', 'RAD:Thermo8:Neutron', 'RAD:Thermo9:Neutron']

dt = 24
FROM_NOW = True
if FROM_NOW:
    final_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    initial_date = (datetime.now() - timedelta(hours=dt)).strftime("%Y-%m-%d %H:%M:%S")
    final_time = datetime.now().strftime('%H:%M:%S')
    initial_time = (datetime.now() - timedelta(hours=dt)).strftime('%H:%M:%S')

else:
    initial_date = '2022-01-09'
    initial_time = '12:00:00'
    final_date = '2022-01-10'
    final_time = '12:00:00'

print(initial_time)


14:45:48


In [61]:
# Recupera dados do archiver
dose_rate = []
url_prefix = 'http://10.0.38.42/retrieval/data/getData.csv?pv='
url_posfix = '&from={}T{}.000Z&to={}T{}.000Z'.format(initial_date,\
      initial_time.replace(':', '%3A'), final_date, final_time.replace(':', '%3A'))
neutron_dfs = {}

for _ in neutron_pvs:
    temp = pd.read_csv('{}{}{}'.format(url_prefix, _, url_posfix),\
         header = None, names = ["secs", "Dose Rate (uSv/h)", "severity", "status", "nanos"])
    temp['date'] = time = pd.to_datetime(temp['secs'], unit='s')
    temp['time (h)'] = (temp['secs'].subtract(temp['secs'][0]))/3600
    temp.drop('nanos', inplace=True, axis=1)
    temp.drop('severity', inplace=True, axis=1)
    temp.drop('status', inplace=True, axis=1)
    temp.drop('secs', inplace=True, axis=1)
    neutron_dfs[_] = temp

In [ ]:
dose = {}
for _ in neutron_pvs:
    dose[_] = integrate.trapz(neutron_dfs[_]['Dose Rate (uSv/h)'].values, neutron_dfs[_]['time (h)'].values)
print(dose)

In [ ]:
fig = plt.figure(figsize=(20,10))
axis = fig.add_subplot(111)
axis.set_title('Taxa de Dose {}'.format('RAD:Thermo9:Neutron'))
axis.set_xlabel('Tempo (s)')
axis.set_ylabel('Taxa de dose')
axis.legend(loc='best')
axis.plot(neutron_dfs['RAD:Thermo9:Neutron']['date'],\
    neutron_dfs['RAD:Thermo9:Neutron']['Dose Rate (uSv/h)'], label = 'RAD:Thermo9:Neutron')
axis.legend()

In [ ]:
fig = plt.figure(figsize=(20,10))
axis = fig.add_subplot(111)
axis.set_title('Taxa de Dose {}'.format('RAD:Thermo9:Neutron'))
axis.set_xlabel('Tempo (s)')
axis.set_ylabel('Taxa de dose')
axis.legend(loc='best')
axis.plot(neutron_dfs['RAD:Thermo9:Neutron']['date'],\
    neutron_dfs['RAD:Thermo9:Neutron']['Dose Rate (uSv/h)'], label = 'RAD:Thermo9:Neutron')
axis.legend()